In [1]:
from pyigrf.igrf import igrf
import numpy as np
import datetime

In [2]:
# Geodetic to Geocentric coordinates transformation
# WGS84 constants
# reference:
# http://earth-info.nga.mil/GandG/publications/tr8350.2/wgs84fin.pdf
a_WGS=6378.137
flatness = 1./298.257223563  # flatenning
b_WGS=a_WGS*(1.-flatness)    # WGS polar radius (semi-minor axis) in km
eccentricity=np.sqrt(a_WGS**2-b_WGS**2)/a_WGS

def geod2geoc(lon,geodlat,h):
    # returns geocentric xyz coordinates (ECEF) in km of a target with
    # latitude       geodlat (rad) --- geodetic
    # longitude      lon (rad)
    # height         h (km above local ellipsoid)
    n=a_WGS / np.sqrt(1.-flatness*(2.-flatness) * np.sin(geodlat)**2.)
    # cartesian geocentric coordinates wrt Greenwich
    x=(n+h)*np.cos(geodlat)*np.cos(lon)
    y=(n+h)*np.cos(geodlat)*np.sin(lon)
    z=(n*(1.-eccentricity**2.)+h)*np.sin(geodlat)   
    
    p   = np.sqrt(x**2. + y**2.)
    geoclat = np.arctan2(z,p)        # geocentric latitude (theta)
    
    return x,y,z,geoclat

## B @ 100 km above PFISR on Feb 09, 2020:
https://amisr.com/amisr/about/about_pfisr/pfisr-specs/

Latitude, Longitude, Altitude: 65.12992 N, 147.47104 W, 213 m

In [3]:
# instantiating IGRF:
igrf0 = igrf(verbose=True)

Using coefficients file: igrf13coeffs.txt
max_n is 13
Last Epoch year is: 2020.0
secular variation: 2020-25


In [4]:
deg = 180/np.pi
geod_lat = 65.12992
lon = -147.47104
ht = 0.213 + 100   #km
dt0 = datetime.datetime(2020,2,9)
yeardays = datetime.datetime(dt0.year,12,31).timetuple().tm_yday
x,y,z,lat = geod2geoc(lon/deg, geod_lat/deg, ht)
doy = dt0.timetuple().tm_yday
year = dt0.year + doy/yeardays

[Bn,Be,Bd,B] = igrf0.igrf_B(year, ht, lon, lat*deg)
Bn,Be,Bd,B

(11149.457507389841, 3392.8408343304177, 52465.00856440682, 53743.82657849618)